In [1]:
#in this version:
# version 2.34
#removed doc api loop that only went through some sheets, as this was missing out some calculations. Now all calcs and default fields should be extracted

In [2]:
import pandas as pd
import bs4 as bs
import zipfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os, re, sys

import win32com.client
from pywintypes import com_error
from os.path import isfile, join

import xml.etree.ElementTree as ET
from lxml import etree as ET

import tableaudocumentapi
from tableaudocumentapi import Workbook

import numpy as np

## Input folder - Find if there is a twbx or twb file in the folder
- if there is a twbx, unzip it to create a twb, then work with this
- if there's only a twb, work with this

In [191]:
input_path = "inputs"
output_path = "outputs"

mypath = "./{}".format(input_path)

In [192]:
#only gets files and not directories within the inputs folder -https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
f = [f for f in os.listdir(mypath) if isfile(join(mypath, f))] 
f

['Superstore_ Sales & Profits.twbx']

In [193]:
def removeSpecialCharFromStr(spstring):
  
    return ''.join(e for e in spstring if e.isalnum())

In [194]:
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] : 
   
    if i[-5:] == '.twbx':
        sp_packagedWorkbook = i[:len(i)-5]
        print(sp_packagedWorkbook)
        packagedWorkbook = removeSpecialCharFromStr(sp_packagedWorkbook)+'.twbx'
        print(packagedWorkbook)
        
        old_file = join(input_path, sp_packagedWorkbook+'.twbx')
        new_file = join(input_path, packagedWorkbook)
        os.rename(old_file, new_file)
        
        with zipfile.ZipFile(input_path+"/"+packagedWorkbook, 'r') as zip_ref:
            zip_ref.extractall(input_path+"/")
    else:
        packagedWorkbook = ""
        
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] :
    
    if i[-4:] == '.twb':
        sp_unpackagedWorkbook = i[:len(i)-4]
        unpackedWorkbook = removeSpecialCharFromStr(sp_unpackagedWorkbook)+'.twb' 
        
        old_file = join(input_path, sp_unpackagedWorkbook+'.twb')
        new_file = join(input_path, unpackedWorkbook)
        os.rename(old_file, new_file)

print('\n')
print('packaged workbook: ' + packagedWorkbook)
print('unpackaged workbook: ' + unpackedWorkbook)

Superstore_ Sales & Profits
SuperstoreSalesProfits.twbx


packaged workbook: SuperstoreSalesProfits.twbx
unpackaged workbook: SuperstoreSalesProfits.twb


In [195]:
tableauFile = input_path+"/"+unpackedWorkbook
tableauFile

'inputs/SuperstoreSalesProfits.twb'

In [196]:
packagedTableauFile = input_path+"/"+packagedWorkbook
packagedTableauFile

'inputs/SuperstoreSalesProfits.twbx'

In [197]:
#substring to be used when naming the exported data, based on the packaged workbook as this is what we download from tableau server
#we can rename the packaged workbook before running the script, giving more control to the naming of the extracted data

tableau_name_substring = packagedWorkbook.replace(".twbx","")[:30]
tableau_name_substring

'SuperstoreSalesProfits'

# Parse xml to get all calculations

In [198]:
tree = ET.parse(tableauFile)
root = tree.getroot()

collator1 = []
calcNames = []
calcCaptions = []

for_findall = ["./datasources/datasource/column", "./worksheets/worksheet/table/view/datasource-dependencies/column"]

for pathy in for_findall:
    for elem in root.findall(pathy):
        #print(elem)

        dict_temp = {}

        if (elem.findall('calculation')) != []:    #only get nodes where there is a calculation
            try:
                #print("caption: " + elem.attrib['caption'])
                dict_temp['caption'] = elem.attrib['caption']
                calcCaptions.append(elem.attrib['caption'])
            except:
                #print('caption: MISSING!')
                dict_temp['caption'] = elem.attrib['name'] #CHANGEBACK #'MISSING'
                calcCaptions.append(elem.attrib['name'])  #CHANGEBACK .append('MISSING')

            dict_temp['datatype'] = elem.attrib['datatype']
            dict_temp['name'] = elem.attrib['name']

            f2 = (elem.attrib['name']).replace(']','')
            f2 = f2.replace('[', '')
            calcNames.append(f2)

            try: #this part evaluates for a parameter
                paramExists = elem.attrib['param-domain-type']
                #print(paramExists)
                dict_temp['isParameter'] = 'yes'
                dict_temp['formula'] = 'NA'

            except: #this part is for calculations only (not parameters)
                dict_temp['isParameter'] = 'no'

                try:
                    for calc in elem.findall('calculation'):
                        dict_temp['formula'] = calc.attrib['formula']
                except:

                    dict_temp['formula'] = 'NA'

            collator1.append(dict_temp)

In [199]:
calcDict = dict(zip(calcNames, calcCaptions))
calcDict

{'Calculation_130534064714207232': 'Info',
 'Calculation_130534065866321923': 'Sales or Profit ',
 'Calculation_696580244257730560': 'Bar Chart Measure Calc',
 'Calculation_696580244265328641': 'SalesProfitColor',
 'Parameter 1': 'Sales or Profit',
 'Parameter 2': 'Bar Chart Measure',
 'Profit (copy)_696580244276445186': 'Profit (copy)',
 'Sales (copy)_696580244277067779': 'Sales (copy)'}

In [200]:
def default_to_friendly_names(formulaList):

    for i in formulaList:
        for tableauName, friendlyName in calcDict.items():
            i['formula'] = (i['formula']).replace(tableauName, friendlyName)
       
    return formulaList

In [201]:
collator1 = default_to_friendly_names(collator1)
collator1[0:2]

[{'caption': 'Sales or Profit',
  'datatype': 'string',
  'formula': 'NA',
  'isParameter': 'yes',
  'name': '[Parameter 1]'},
 {'caption': 'Bar Chart Measure',
  'datatype': 'string',
  'formula': 'NA',
  'isParameter': 'yes',
  'name': '[Parameter 2]'}]

In [202]:
df = pd.DataFrame(collator1)

df.columns = ['CalculationName', 'DataType', 'Formula', 'isParameter', 'RawName']

df = df.drop_duplicates()

df = df.sort_values(by=['isParameter','CalculationName'])
df = df.reset_index(drop=True)
df

,CalculationName,DataType,Formula,isParameter,RawName
0,Bar Chart Measure Calc,string,"CASE [Parameters].[Bar Chart Measure]\nWHEN ""R...",no,[Calculation_696580244257730560]
1,Info,string,"""""",no,[Calculation_130534064714207232]
2,Profit (copy),real,[Profit],no,[Profit (copy)_696580244276445186]
3,Sales (copy),real,[Sales],no,[Sales (copy)_696580244277067779]
4,Sales or Profit,real,"IF [Parameters].[Sales or Profit] = ""Sales"" ...",no,[Calculation_130534065866321923]
5,SalesProfitColor,integer,"IF [Parameters].[Sales or Profit] = ""Sales"" Th...",no,[Calculation_696580244265328641]
6,Bar Chart Measure,string,NA,yes,[Parameter 2]
7,Sales or Profit,string,NA,yes,[Parameter 1]


# Getting all filters for all worksheets

In [204]:
# for worskheet in root.findall("./worksheets/worksheet"):
#     for filt in worskheet.findall('table/view/filter'):
#         print(filt)

In [205]:
tree = ET.parse(tableauFile)
root = tree.getroot()

filters_in_sheet = []
context = []
collatelist = []

for worskheet in root.findall("./worksheets/worksheet"):
    
    tempdict = {}
    c = 0
    
    for filt in worskheet.findall('table/view/filter'):

        calcfromfilter = filt.attrib['column']        
        pat = '(?<=\:)(.*?)(?=\:)' 
        string_cleaned = calcfromfilter.split('].[')[1].replace(']','')
        
        tempdict['field'] = calcfromfilter
        tempdict['formula'] = calcfromfilter
        tempdict['counter'] = c
        tempdict['sheetname'] = worskheet.attrib['name']
        
        try:
            st1 = re.findall(pat,string_cleaned)[0]
            tempdict['field'] = st1
            tempdict['formula'] = st1
            collatelist.append(tempdict)
            
        except:
            st2 = string_cleaned.replace(':','')
            tempdict['field'] = st2
            tempdict['formula'] = st2
            collatelist.append(tempdict)

        try:
            tempdict['context'] = filt.attrib['context']
        except:
            tempdict['context'] = 'False'
           
        c = c + 1
        tempdict = {}
    
collatelist[0:2]

[{'context': 'False',
  'counter': 0,
  'field': 'Action (Category,Region,Sub-Category)',
  'formula': 'Action (Category,Region,Sub-Category)',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)'},
 {'context': 'False',
  'counter': 1,
  'field': 'Action (Country/Region,State)',
  'formula': 'Action (Country/Region,State)',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)'}]

In [206]:
collatelist = default_to_friendly_names(collatelist)
collatelist[0:2]

[{'context': 'False',
  'counter': 0,
  'field': 'Action (Category,Region,Sub-Category)',
  'formula': 'Action (Category,Region,Sub-Category)',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)'},
 {'context': 'False',
  'counter': 1,
  'field': 'Action (Country/Region,State)',
  'formula': 'Action (Country/Region,State)',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)'}]

In [207]:
try: 
    df1 = pd.DataFrame(collatelist)

    df1 = df1[['sheetname', 'formula', 'context', 'field']]
    df1.columns = ['Sheet Name', 'FilterField', 'Context filter', 'FilterField_RawName']

    print(df1.head(2))
except:
    print('error with df1')

                                     Sheet Name  \
0  Bar_Sum of Profit by Category and Region (2)   
1  Bar_Sum of Profit by Category and Region (2)   

                             FilterField Context filter  \
0  Action (Category,Region,Sub-Category)          False   
1          Action (Country/Region,State)          False   

                     FilterField_RawName  
0  Action (Category,Region,Sub-Category)  
1          Action (Country/Region,State)  


# Extracting rows and cols for each sheet

In [208]:
collecteddata = []

for worksheet in root.findall("./worksheets/worksheet"):

    argumentstopass = ['rows', 'cols']
    
    for i in argumentstopass:   
    
        internaldict = {}

        internaldict['sheetname'] = worksheet.attrib['name']
        internaldict['type'] = i
        
        formulahere = worksheet.findall('table/'+i)[0].text
        internaldict['formula'] = formulahere
        
        collecteddata.append(internaldict)
    
collecteddata[0:2]

[{'formula': '[federated.0ad0ou31g7vofj105j82y1lwnwht].[none:Calculation_696580244257730560:nk]',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)',
  'type': 'rows'},
 {'formula': '[federated.0ad0ou31g7vofj105j82y1lwnwht].[usr:Calculation_130534065866321923:qk]',
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)',
  'type': 'cols'}]

In [209]:
for i in collecteddata:

    try:
        pattern = '\:.*?\:'
        pat = '(?<=\:)(.*?)(?=\:)'

        calculationsWithColon = re.findall(pattern,i['formula']) 
        calcsWithoutColon = []

        for n in calculationsWithColon:
            oneCalcWithoutColon = re.findall(pat,n)[0]

            calcsWithoutColon.append(oneCalcWithoutColon)

        i['extracted formulas'] = calcsWithoutColon
        
    except:
        i['extracted formulas'] = []
             
    newcalcs = []
    formulas_to_process = i['extracted formulas']
    
    for n in formulas_to_process:
           
        for tableauName, friendlyName in calcDict.items():
            
            n = n.replace(tableauName, friendlyName)
            
        newcalcs.append(n)
        
    i['processed formulas'] = newcalcs

collecteddata

[{'extracted formulas': ['Calculation_696580244257730560'],
  'formula': '[federated.0ad0ou31g7vofj105j82y1lwnwht].[none:Calculation_696580244257730560:nk]',
  'processed formulas': ['Bar Chart Measure Calc'],
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)',
  'type': 'rows'},
 {'extracted formulas': ['Calculation_130534065866321923'],
  'formula': '[federated.0ad0ou31g7vofj105j82y1lwnwht].[usr:Calculation_130534065866321923:qk]',
  'processed formulas': ['Sales or Profit '],
  'sheetname': 'Bar_Sum of Profit by Category and Region (2)',
  'type': 'cols'},
 {'extracted formulas': ['Region'],
  'formula': '[federated.0ad0ou31g7vofj105j82y1lwnwht].[none:Region:nk]',
  'processed formulas': ['Region'],
  'sheetname': 'Crosstab_Sum of Profit by Category and Region',
  'type': 'rows'},
 {'extracted formulas': ['Category', 'Sub-Category'],
  'formula': '([federated.0ad0ou31g7vofj105j82y1lwnwht].[none:Category:nk] / [federated.0ad0ou31g7vofj105j82y1lwnwht].[none:Sub-Category:nk]

In [210]:
df2 = pd.DataFrame(collecteddata)
df2 = df2.drop(columns=['formula', 'extracted formulas'])
df2 = df2.pivot(index='sheetname', columns='type', values='processed formulas')
df2 = df2.reset_index()
df2.head(2)

type,sheetname,cols,rows
0,Bar_Sum of Profit by Category and Region (2),[Sales or Profit ],[Bar Chart Measure Calc]
1,Crosstab_Sum of Profit by Category and Region,"[Category, Sub-Category]",[Region]


# Doc API

# All default fields - DOC API

In [211]:
packagedTableauFile

'inputs/SuperstoreSalesProfits.twbx'

In [212]:
#get all fields in workbook
sourceTWBX = Workbook(packagedTableauFile)

collator = []
calcID = []
calcID2 = []
calcNames = []

c = 0

worksheets = sourceTWBX.worksheets

#for worksheet in worksheets: #see if this has to be marked out or not
    
for datasource in sourceTWBX.datasources:

    for count, field in enumerate(datasource.fields.values()):

                #if worksheet in field.worksheets: #removed this part so all fields are listed,as otherwise some fields were missed out

            dict_temp = {}
            dict_temp['counter'] = c
            dict_temp['worksheet'] = worksheet
            dict_temp['datasource_name'] = datasource.name
            dict_temp['field_WHOLE'] = field
            dict_temp['field_name'] = field.name
            dict_temp['field_caption'] = field.caption
            dict_temp['field_calculation'] = field.calculation
            dict_temp['field_id'] = field.id
            dict_temp['field_datatype'] = field.datatype


            if not(isinstance(field.calculation, type(None))):
                calcID.append(field.id)
                calcNames.append(field.name)

                f2 = (field.id).replace(']','')
                f2 = f2.replace('[', '')
                calcID2.append(f2)

            c = c + 1

            collator.append(dict_temp)

local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias

In [213]:
calcDict = dict(zip(calcID, calcNames))
calcDict2 = dict(zip(calcID2, calcNames)) #raw fields without any []

def default_to_friendly_names2(formulaList,fieldToConvert, dictToUse):

    for i in formulaList:
        for tableauName, friendlyName in dictToUse.items():
            try:
                i[fieldToConvert] = (i[fieldToConvert]).replace(tableauName, friendlyName)
            except:
                a = 0
       
    return formulaList

In [214]:
def f(row):
    if row['field_calculation'] == None:
        val = 'Datasource field'
    else:
        val = 'Calculated field'
    return val

In [215]:
default_to_friendly_names2(collator,'field_calculation',calcDict)

df_API_all = pd.DataFrame(collator)
df_API_all['field_type'] = df_API_all.apply(f, axis=1)

df_API_all.head()

,counter,datasource_name,field_WHOLE,field_calculation,field_caption,field_datatype,field_id,field_name,worksheet,field_type
0,0,Parameters,------ FIELD PROFIT (Sales or Profit/PROFIT/[P...,"""Profit""",Sales or Profit,string,[Parameter 1],PROFIT,"[[[<Element datasources at 0x541fc38>, <Elemen...",Calculated field
1,1,Parameters,------ FIELD REGION (Bar Chart Measure/REGION/...,"""REGION""",Bar Chart Measure,string,[Parameter 2],REGION,"[[[<Element datasources at 0x541fc38>, <Elemen...",Calculated field
2,2,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Row ID (None/None/Row ID): None(t...,None,None,integer,Row ID,Row ID,"[[[<Element datasources at 0x541fc38>, <Elemen...",Datasource field
3,3,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Order ID (None/None/Order ID): No...,None,None,string,Order ID,Order ID,"[[[<Element datasources at 0x541fc38>, <Elemen...",Datasource field
4,4,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Order Date (None/None/Order Date)...,None,None,date,Order Date,Order Date,"[[[<Element datasources at 0x541fc38>, <Elemen...",Datasource field


In [216]:
df_defaultFields = df_API_all[df_API_all['field_type'] == 'Datasource field'][['field_id', 'field_caption','field_datatype', 'datasource_name']].drop_duplicates().copy()

df_defaultFields['prefOrder'] = np.where(df_defaultFields['field_caption'].isnull(), 0, 1)
df_defaultFields['field_id2'] = df_defaultFields['field_id'].str.replace('[','')
df_defaultFields['field_id2'] = df_defaultFields['field_id2'].str.replace(']','')

df_defaultFields = df_defaultFields.sort_values(by = ['field_id2'])
#https://stackoverflow.com/questions/63271050/use-drop-duplicates-in-pandas-df-but-choose-keep-column-based-on-a-preference-li
preference_list=[1,0]

df_defaultFields["prefOrder"] = pd.Categorical(df_defaultFields["prefOrder"], categories=preference_list, ordered=True)

df_defaultFields = df_defaultFields.sort_values(["field_id2","prefOrder"]).drop_duplicates("field_id2")
df_defaultFields = df_defaultFields.drop('prefOrder', axis=1)
df_defaultFields = df_defaultFields.drop('field_id2', axis=1)
df_defaultFields.head(2)

,field_id,field_caption,field_datatype,datasource_name
28,[Category],None,string,federated.0ad0ou31g7vofj105j82y1lwnwht
29,[City],None,string,federated.0ad0ou31g7vofj105j82y1lwnwht


# API Calculated fields - check if any missing from xml

In [217]:
df_API_all

,counter,datasource_name,field_WHOLE,field_calculation,field_caption,field_datatype,field_id,field_name,worksheet,field_type
0,0,Parameters,------ FIELD PROFIT (Sales or Profit/PROFIT/[P...,"""Profit""",Sales or Profit,string,[Parameter 1],PROFIT,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Calculated field
1,1,Parameters,------ FIELD REGION (Bar Chart Measure/REGION/...,"""REGION""",Bar Chart Measure,string,[Parameter 2],REGION,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Calculated field
2,2,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Row ID (None/None/Row ID): None(t...,None,None,integer,Row ID,Row ID,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
3,3,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Order ID (None/None/Order ID): No...,None,None,string,Order ID,Order ID,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
4,4,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Order Date (None/None/Order Date)...,None,None,date,Order Date,Order Date,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
5,5,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Ship Date (None/None/Ship Date): ...,None,None,date,Ship Date,Ship Date,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
6,6,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Ship Mode (None/None/Ship Mode): ...,None,None,string,Ship Mode,Ship Mode,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
7,7,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Customer ID (None/None/Customer I...,None,None,string,Customer ID,Customer ID,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
8,8,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Customer Name (None/None/Customer...,None,None,string,Customer Name,Customer Name,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field
9,9,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Segment (None/None/Segment): None...,None,None,string,Segment,Segment,"[[[<Element datasources at 0x53fbf08>, <Elemen...",Datasource field


In [218]:
colsToUse = ['field_id', 'field_name', 'field_calculation', 'field_caption','field_datatype', 'datasource_name' ]
df_API_all[colsToUse][df_API_all['field_type']!='Datasource field'].drop_duplicates()#.copy()

,field_id,field_name,field_calculation,field_caption,field_datatype,datasource_name
0,[Parameter 1],PROFIT,"""Profit""",Sales or Profit,string,Parameters
1,[Parameter 2],REGION,"""REGION""",Bar Chart Measure,string,Parameters
24,[Calculation_130534064714207232],Info,"""""",Info,string,federated.0ad0ou31g7vofj105j82y1lwnwht
25,[Calculation_130534065866321923],Sales or Profit,"IF [Parameters].PROFIT = ""Sales"" then SUM(Sa...",Sales or Profit,real,federated.0ad0ou31g7vofj105j82y1lwnwht
26,[Calculation_696580244257730560],Bar Chart Measure Calc,"CASE [Parameters].REGION\nWHEN ""REGION"" THEN [...",Bar Chart Measure Calc,string,federated.0ad0ou31g7vofj105j82y1lwnwht
27,[Calculation_696580244265328641],SalesProfitColor,"IF [Parameters].PROFIT = ""Sales"" Then 1\nELSEI...",SalesProfitColor,integer,federated.0ad0ou31g7vofj105j82y1lwnwht
33,[Profit (copy)_696580244276445186],Profit (copy),[Profit],Profit (copy),real,federated.0ad0ou31g7vofj105j82y1lwnwht
37,[Sales (copy)_696580244277067779],Sales (copy),[Sales],Sales (copy),real,federated.0ad0ou31g7vofj105j82y1lwnwht
41,[Parameter 1],PROFIT,"""Profit""",Sales or Profit,string,federated.0ad0ou31g7vofj105j82y1lwnwht
42,[Parameter 2],REGION,"""REGION""",Bar Chart Measure,string,federated.0ad0ou31g7vofj105j82y1lwnwht


# Parameters

In [219]:
dfAPIParameters = df_API_all[colsToUse][df_API_all['datasource_name']=='Parameters'].drop_duplicates().copy()

dfAPIParameters

,field_id,field_name,field_calculation,field_caption,field_datatype,datasource_name
0,[Parameter 1],PROFIT,"""Profit""",Sales or Profit,string,Parameters
1,[Parameter 2],REGION,"""REGION""",Bar Chart Measure,string,Parameters


In [239]:
df = df.merge(dfAPIParameters[['field_id','field_calculation']], left_on='RawName', right_on = 'field_id', how='left')

df["Formula"] = np.where(df["Formula"] == "NA", df['field_calculation'], df["Formula"])
df = df.drop(columns=['field_id', 'field_calculation'])
df

,CalculationName,DataType,Formula,isParameter,RawName
0,Bar Chart Measure Calc,string,"CASE [Parameters].[Bar Chart Measure]\nWHEN ""R...",no,[Calculation_696580244257730560]
1,Info,string,"""""",no,[Calculation_130534064714207232]
2,Profit (copy),real,[Profit],no,[Profit (copy)_696580244276445186]
3,Sales (copy),real,[Sales],no,[Sales (copy)_696580244277067779]
4,Sales or Profit,real,"IF [Parameters].[Sales or Profit] = ""Sales"" ...",no,[Calculation_130534065866321923]
5,SalesProfitColor,integer,"IF [Parameters].[Sales or Profit] = ""Sales"" Th...",no,[Calculation_696580244265328641]
6,Bar Chart Measure,string,"""REGION""",yes,[Parameter 2]
7,Sales or Profit,string,"""Profit""",yes,[Parameter 1]


# Sheet - all field dependencies, not just the explicitly used fields

In [220]:
#df_api_insheet
sourceTWBX = Workbook(packagedTableauFile)

collator_sheet_dependencies = []

c = 0

worksheets = sourceTWBX.worksheets

for worksheet in worksheets:
    
    for datasource in sourceTWBX.datasources:
       
        for count, field in enumerate(datasource.fields.values()):
            
            if worksheet in field.worksheets: #to see if only fields that appear in sheets are listed, else last df is too large
                
                dict_temp = {}
                dict_temp['counter'] = c
                dict_temp['worksheet'] = worksheet
                dict_temp['datasource_name'] = datasource.name
                dict_temp['field_WHOLE'] = field
                dict_temp['field_name'] = field.name
                dict_temp['field_caption'] = field.caption
                dict_temp['field_calculation'] = field.calculation
                dict_temp['field_id'] = field.id
                dict_temp['field_datatype'] = field.datatype
                
                c = c + 1
                
                collator_sheet_dependencies.append(dict_temp)

local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias

In [221]:
#default_to_friendly_names2(collator_sheet_dependencies, 'field_calculation',calcDict)

df_api_insheet = pd.DataFrame(collator_sheet_dependencies)
df_api_insheet['field_type'] = df_api_insheet.apply(f, axis=1)
df_api_insheet.head()

,counter,datasource_name,field_WHOLE,field_calculation,field_caption,field_datatype,field_id,field_name,worksheet,field_type
0,0,Parameters,------ FIELD PROFIT (Sales or Profit/PROFIT/[P...,"""Profit""",Sales or Profit,string,[Parameter 1],PROFIT,Bar_Sum of Profit by Category and Region (2),Calculated field
1,1,Parameters,------ FIELD REGION (Bar Chart Measure/REGION/...,"""REGION""",Bar Chart Measure,string,[Parameter 2],REGION,Bar_Sum of Profit by Category and Region (2),Calculated field
2,2,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Sales or Profit (Sales or Profit...,"IF [Parameters].[Parameter 1] = ""Sales"" then...",Sales or Profit,real,[Calculation_130534065866321923],Sales or Profit,Bar_Sum of Profit by Category and Region (2),Calculated field
3,3,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD Bar Chart Measure Calc (Bar Chart...,"CASE [Parameters].[Parameter 2]\nWHEN ""REGION""...",Bar Chart Measure Calc,string,[Calculation_696580244257730560],Bar Chart Measure Calc,Bar_Sum of Profit by Category and Region (2),Calculated field
4,4,federated.0ad0ou31g7vofj105j82y1lwnwht,------ FIELD SalesProfitColor (SalesProfitColo...,"IF [Parameters].[Parameter 1] = ""Sales"" Then 1...",SalesProfitColor,integer,[Calculation_696580244265328641],SalesProfitColor,Bar_Sum of Profit by Category and Region (2),Calculated field


In [222]:
df_sheetDependencies = df_api_insheet.copy()
preference_list=[1,0]

df_sheetDependencies['prefOrder'] = np.where(df_sheetDependencies['field_caption'].isnull(), 0, 1)

df_sheetDependencies['field_id2'] = df_sheetDependencies['field_id'].str.replace('[','')
df_sheetDependencies['field_id2'] = df_sheetDependencies['field_id2'].str.replace(']','')

df_sheetDependencies["prefOrder"] = pd.Categorical(df_sheetDependencies["prefOrder"], categories=preference_list, ordered=True)
df_sheetDependencies = df_sheetDependencies.sort_values(["field_id2",\
                                                         "prefOrder"]).drop_duplicates(subset=["field_id2", "worksheet"])

df_sheetDependencies = df_sheetDependencies.drop(\
                                columns=['prefOrder', 'field_id2', 'counter', 'field_caption', 'field_WHOLE', \
                                         'field_calculation', 'field_id'])

df_sheetDependencies = df_sheetDependencies[['worksheet', 'field_name', 'field_datatype', \
                                             'field_type', 'datasource_name']].sort_values(by = ['worksheet', 'field_type', 'field_name'])
df_sheetDependencies.head()

,worksheet,field_name,field_datatype,field_type,datasource_name
3,Bar_Sum of Profit by Category and Region (2),Bar Chart Measure Calc,string,Calculated field,federated.0ad0ou31g7vofj105j82y1lwnwht
0,Bar_Sum of Profit by Category and Region (2),PROFIT,string,Calculated field,Parameters
1,Bar_Sum of Profit by Category and Region (2),REGION,string,Calculated field,Parameters
2,Bar_Sum of Profit by Category and Region (2),Sales or Profit,real,Calculated field,federated.0ad0ou31g7vofj105j82y1lwnwht
4,Bar_Sum of Profit by Category and Region (2),SalesProfitColor,integer,Calculated field,federated.0ad0ou31g7vofj105j82y1lwnwht


# General workbook description

In [223]:
sourceTWBX = Workbook(packagedTableauFile)

local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias

In [224]:
collate_list = []

for dash in sourceTWBX.dashboards:
    dicti = {}
    
    dicti['type'] = 'dashboard'
  #  print(format(dash))
    dicti['name'] = format(dash)
   
    collate_list.append(dicti)
    
for data in sourceTWBX.datasources:
    dicti = {}
    
    dicti['type'] = 'datasource'
    dicti['name'] = format(data.name)
   # print(format(data.name))
   
    collate_list.append(dicti)
    
for data in sourceTWBX.worksheets:
    dicti = {}
    
    dicti['type'] = 'sheet'
    dicti['name'] = format(data)
   # print(format(data))
    
    collate_list.append(dicti)

In [225]:
df_workbookdec = pd.DataFrame(collate_list)
df_workbookdec = df_workbookdec[['type', 'name']]
df_workbookdec.head(2)

,type,name
0,dashboard,Superstore Sales Analysis
1,datasource,Parameters


In [226]:
df_workbookdec_counts = df_workbookdec.groupby(['type']).count().reset_index()
df_workbookdec_counts

,type,name
0,dashboard,1
1,datasource,2
2,sheet,10


## Generating an excel file from a df (so the excel rows/cols can be formatted), then turning the excel into a pdf

In [227]:
#load env variable with full path to folder, as otherwise the pdf creation fails

with open("archive/path_string.txt", "r") as f:
    os.environ["path_string"] = f.read()
    
path_string = os.environ.get('path_string', 'Not Set')
path_string = r"{}".format(path_string)    #must create literal ending in tableauCalculationExport\{}

- Loading the file names and output locations for the excel and pdfs to be produced

In [228]:
name_to_use = tableau_name_substring    

newFileName = 'outputs\{}'.format(name_to_use)
excelName = newFileName + ".xlsx"
pdfName = newFileName + ".pdf"
print(pdfName)

excel_path = path_string.format(excelName)
path_to_pdf = path_string.format(pdfName)

outputs\SuperstoreSalesProfits.pdf


- Functions to format the excel files

In [229]:
#colors to be used in each sheet
c1 = '#f4dfa4'
c2 = '#ffc8b3'
c3 = '#fff0b3'
c4 = '#d5dfb9'
c5 = '#d1c5d3'
c6 = '#bfd9d7'

In [230]:
def mainCol(colNumber, color):
    format_mainCol = workbook.add_format({'text_wrap': True, 'bold': True})
    format_mainCol.set_align('vcenter')
    format_mainCol.set_bg_color(color)
    format_mainCol.set_border(1)
    worksheet.set_column(colNumber,colNumber,20,format_mainCol)
    return worksheet

In [231]:
def normalCol(colNumber, colWidth):
    format2 = workbook.add_format({'text_wrap': True})
    format2.set_align('vcenter')
    format2.set_border(1)
    worksheet.set_column(colNumber,colNumber,colWidth,format2)
    return worksheet

- Creation of excel file

In [232]:
#modify this part if you want to add more information/dfs to be saved as a separate sheet in excel

dfs_to_use = [{'excelSheetTitle': 'Dashboard, datasource and sheet details', 'df_to_use':df_workbookdec, 'mainColWidth':'' , 
               'normalColWidth': [30], 'sheetName': 'GeneralDetails', 'footer': 'Data_1', 'papersize':9, 'color': c1} , 
              
              {'excelSheetTitle': 'Overall counts of dashboards, datasources and sheets', 'df_to_use':df_workbookdec_counts, 'mainColWidth':'' , 
               'normalColWidth': [10], 'sheetName': 'GeneralCounts', 'footer': 'Data_2', 'papersize':9, 'color': c1},
              
              {'excelSheetTitle': 'Default fields from all datasources', 'df_to_use':df_defaultFields, 'mainColWidth':'' , 
               'normalColWidth': [20,20,40], 'sheetName': 'DefaultFields', 'footer': 'Data_3', 'papersize':9, 'color': c2},
              
              {'excelSheetTitle': 'Calculated fields and parameters', 'df_to_use':df, 'mainColWidth':'' , 
               'normalColWidth': [10,50,10,20], 'sheetName': 'CalculatedFields', 'footer': 'Data_4', 'papersize':9, 'color': c3},
              
              {'excelSheetTitle': 'Filters used in each sheet', 'df_to_use':df1, 'mainColWidth':'' , 
               'normalColWidth': [20,20,40], 'sheetName': 'Filters', 'footer': 'Data_5', 'papersize':9, 'color': c4},
              
              {'excelSheetTitle': 'Metrics used in Columns and Rows, for each sheet', 'df_to_use':df2, 'mainColWidth':'' , 
               'normalColWidth': [30,40], 'sheetName': 'RowsAndCols', 'footer': 'Data_6', 'papersize':9, 'color': c5},
              
              {'excelSheetTitle': 'Sheet dependencies on default fields, calculated fields and parameters', 'df_to_use':df_sheetDependencies, 'mainColWidth':'' , 
               'normalColWidth': [30,15,25,30], 'sheetName': 'SheetDependencies', 'footer': 'Data_7', 'papersize':8, 'color': c6}
             ]

#papersize: a3 = 8, a4 = 9

In [233]:
writer = pd.ExcelWriter(excelName, engine = 'xlsxwriter')

#code to create each sheet in excel, with the specified df and formatting each sheet as per requirements
#also adds a header and footer to each sheet
#all the info to be replaced below (ie. for each df) comes form the dfs_to_use list of dictionaries

for x in dfs_to_use:
    excelSheetTitle = x['excelSheetTitle']
    df_to_use = x['df_to_use']
    normalColWidth = x['normalColWidth']
    sheetName = x['sheetName']
    papersize = x['papersize']
    footer = x['footer']
    color = x['color']

    df_to_use.to_excel(writer, sheet_name = sheetName, index=False)
    
    workbook=writer.book
    worksheet = writer.sheets[sheetName]

    worksheet = mainCol(0, color)
    
    ws = 1
    for i in normalColWidth:
        worksheet = normalCol(ws,i)
        ws = ws + 1

    worksheet.set_paper(papersize) # a4
    worksheet.fit_to_pages(1,0)    # fit to 1 page wide, n long
    worksheet.repeat_rows(0)       # repeat the first row
    
    header_x = '&C&"Arial,Bold"&10{}'.format(excelSheetTitle)
    footer_x = '&L{}&CPage &P of &N'.format(footer)

    worksheet.set_header(header_x)
    worksheet.set_footer(footer_x)

writer.save()

- Creation of pdf from excel file

In [234]:
#this creates an index to list each excel sheet, based on the number of sheets that were created before

for_ws_index_list = []
for i in range(len(dfs_to_use)):
    for_ws_index_list.append(i+1)

In [235]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = excel.Workbooks.Open(excel_path)

#print all the excel sheets into a single pdf
ws_index_list = for_ws_index_list
wb.Worksheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb.Close()
excel.Quit()